# モジュールインポート

In [56]:
### 基本モジュール
import pandas as pd
import numpy as np
from datetime import datetime

### スクレイピング
from bs4 import BeautifulSoup
import requests

# スクレイピング
- https://kabuoji3.com/ より取得

## テスト動作

### 初期設定＆htmlの取得

In [57]:
### 初期設定
year = 2019            # 取得年
stock_num = 6577       # 銘柄コード
url = "https://kabuoji3.com/stock/{}/{}/".format(stock_num, year)    # urlの指定
headers = {    # ユーザエージェント情報　自分が使っているプログラムや機械の情報を示す必要がある　
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
}
soup = BeautifulSoup(requests.get(url, headers=headers).content, "html.parser")       # HTMLの取得

### DataFrameに格納する

In [58]:
tag_tr = soup.find_all("tr")          # <tr>タグの中身を取得する
head = [h.text for h in tag_tr[0].find_all("th")]    #<tr>タグの一つ目の<th>タグを取得する

### テーブルデータの取得
data = []
for i in range(1,len(tag_tr)):
    data.append([d.text for d in tag_tr[i].find_all("td")])

df = pd.DataFrame(data, columns=head)      # dataframeに格納する
df.columns

Index(['日付', '始値', '高値', '安値', '終値', '出来高', '終値調整'], dtype='object')

### 型変換

In [59]:
### float型への変換
col = ["始値", "高値", "安値", "終値", "出来高", "終値調整"]
for c in col:
    df[c] = df[c].astype("float")
    
### datetimeへの変換
df["日付"] = [datetime.strptime(i, "%Y-%m-%d") for i in df["日付"]]

print(df["日付"].dtypes)

datetime64[ns]


### 複数年のデータの取得

In [65]:
dfs = []
year_list = [2018, 2019]
stock_num = 6577       # 銘柄コード

for year in year_list:
    ### HTMLの取得
    url = "https://kabuoji3.com/stock/{}/{}/".format(stock_num, year)    # urlの指定
    headers = {    # ユーザエージェント情報　自分が使っているプログラムや機械の情報を示す必要がある　
                "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36"
              }
    soup = BeautifulSoup(requests.get(url, headers=headers).content, "html.parser")       # HTMLの取得
    
    ### DataFrameに格納する
    tag_tr = soup.find_all("tr")          # <tr>タグの中身を取得する
    head = [h.text for h in tag_tr[0].find_all("th")]    #<tr>タグの一つ目の<th>タグを取得する

    ### テーブルデータの取得
    data = []
    for i in range(1,len(tag_tr)):
        data.append([d.text for d in tag_tr[i].find_all("td")])

    df = pd.DataFrame(data, columns=head)      # dataframeに格納する
    
    ### 型変換
    ### float型への変換
    col = ["始値", "高値", "安値", "終値", "出来高", "終値調整"]
    for c in col:
        df[c] = df[c].astype("float")
    ### datetimeへの変換
    df["日付"] = [datetime.strptime(i, "%Y-%m-%d") for i in df["日付"]]
    
    dfs.append(df)
    
### DataFrameの結合
data = pd.concat(dfs, axis=0)
data = data.reset_index(drop=True)

In [69]:
! pip list

Package                           Version  
--------------------------------- ---------
asn1crypto                        0.24.0   
attrs                             19.1.0   
backcall                          0.1.0    
beautifulsoup4                    4.8.2    
bleach                            3.1.0    
boto                              2.49.0   
boto3                             1.9.159  
botocore                          1.12.159 
certifi                           2019.6.16
cffi                              1.12.3   
chardet                           3.0.4    
colorama                          0.4.1    
convertdate                       2.1.3    
cryptography                      2.7      
cycler                            0.10.0   
Cython                            0.29.13  
decorator                         4.4.0    
defusedxml                        0.5.0    
docutils                          0.14     
entrypoints                       0.3      
ephem                           

## 本番